In [ ]:
import re

def extract_amar(text):
    if pd.isna(text): return ""
    text = text.lower()

    # Ambil semua kalimat
    sentences = re.split(r'[.\n]', text)

    for s in sentences:
        if re.search(r'(menyatakan|menjatuhkan|mengadili|amar putusan|putusan|terdakwa.*(dipidana|dijatuhi))', s):
            if len(s.strip()) > 30:
                return s.strip()

    return ""


df['solusi'] = df['text_full'].apply(extract_amar)

# Simpan dict {case_id: solusi}
case_solutions = dict(zip(df['case_id'], df['solusi']))


In [ ]:
from collections import Counter

def predict_outcome(query: str, k: int = 5) -> str:
    top_k = retrieve(query, k)
    solutions = [case_solutions.get(c, "").strip() for c in top_k if case_solutions.get(c)]

    if not solutions:
        return "Tidak ditemukan solusi"

    # Majority vote
    common = Counter(solutions).most_common(1)
    return common[0][0] if common else "Tidak ditemukan solusi"


In [ ]:
queries = [
    {"id": 1, "query": "penipuan dana nasabah oleh teller bank"},
    {"id": 2, "query": "penggelapan anggaran kredit koperasi"},
    {"id": 3, "query": "korupsi direktur bank swasta"},
    {"id": 4, "query": "pemalsuan dokumen rekening"},
    {"id": 5, "query": "penyaluran kredit fiktif oleh manajer bank"},
]

results = []

for q in queries:
    pred = predict_outcome(q['query'], k=5)
    top_cases = retrieve(q['query'], k=5)
    results.append({
        "query_id": q["id"],
        "query": q["query"],
        "predicted_solution": pred,
        "top_5_case_ids": top_cases
    })
    print(f"🟢 Query {q['id']}: {q['query']}\n🎯 Prediksi Amar: {pred}\n📄 Top-5: {top_cases}\n")


🟢 Query 1: penipuan dana nasabah oleh teller bank
🎯 Prediksi Amar: memeriksa dan mengadili perkara perdata tertentu pada tingkat pertama dalam
📄 Top-5: [100, 104, 150, 144, 141]

🟢 Query 2: penggelapan anggaran kredit koperasi
🎯 Prediksi Amar: menyatakan terdakwa shinta irbianti telah terbukti secara sah dan
📄 Top-5: [100, 131, 138, 123, 29]

🟢 Query 3: korupsi direktur bank swasta
🎯 Prediksi Amar: memeriksa dan mengadili perkara perdata tertentu pada tingkat pertama dalam
📄 Top-5: [117, 145, 142, 110, 128]

🟢 Query 4: pemalsuan dokumen rekening
🎯 Prediksi Amar: menyatakan terdakwa focksy rapar alias focksy, terbukti
📄 Top-5: [42, 121, 79, 100, 104]

🟢 Query 5: penyaluran kredit fiktif oleh manajer bank
🎯 Prediksi Amar: memeriksa dan mengadili perkara perdata tertentu pada tingkat pertama dalam
📄 Top-5: [100, 150, 141, 144, 127]



In [ ]:
pred_df = pd.DataFrame(results)
result_path = "/content/drive/MyDrive/cbr_banking/data/results/predictions.csv"
Path(result_path).parent.mkdir(parents=True, exist_ok=True)
pred_df.to_csv(result_path, index=False)